In [ ]:
import streamlink
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image
import imagehash

In [ ]:
taxi_color = [12,196,214]
taxis = []

In [ ]:
possible_duplicates = {}

class dupinstance:
    def __init__(self, img):
        self.img   = img
        self.birth = time.time()

In [ ]:
# misc functions

def cv2resize(img, scale_percent):
    
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    return resized


def compare(enum, im1, im2):
    
    nim2 = cv2.resize(im2, (im1.shape[1],im1.shape[0]))
    
    im_v = cv2.hconcat([im1, nim2])
    
    diff_ = np.sum(np.square(nim2-im1))/(im1.shape[0]*im1.shape[2])
    diff  = imagehash.average_hash(Image.fromarray(nim2))-imagehash.average_hash(Image.fromarray(im1))
    
    #cv2.imwrite("pairs/comparison/c{}_{}.jpg".format(repr(diff).replace(".", "x"), repr(int(diff_//1)).replace(".", "x")), im_v)
    
    if diff_ < 6400 and diff < 15:
        #cv2.imwrite("pairs/similar/d{:06d}_{}_{}.jpg".format(int(diff//1), int(time.time()//1), enum), im_v)
        return "similar"
    elif diff_ < 6900 and diff < 21:
        #cv2.imwrite("pairs/different/d{:06d}_{}_{}.jpg".format(int(diff//1), int(time.time()//1), enum), im_v)
        return "ambigous"
    else:
        return "different"
    
    
def taxi_score(img, threshold = 500):
    
    count  = 0
    color_br = np.ones(img.shape)*taxi_color

    for row in range (img.shape[0]):
        for col in range (img.shape[1]):
            if np.mean(np.square(img[row, col] - color_br[row, col])) < threshold:
                count+=1

    return count

In [ ]:
from streamlink import Streamlink

session = Streamlink()

session.set_option("http-ssl-verify", False)

# https://livestream.ibb.gov.tr/ibb_live/istiklalcad2hq.stream/Playlist.m3u8
# https://livestream.ibb.gov.tr/ibb_live/misircarsisihq.stream/chunklist_w289409329.m3u8
# https://livestream.ibb.gov.tr/ibb_live/istiklalcadhq.stream/Playlist.m3u8

streams = session.streams("https://livestream.ibb.gov.tr/ibb_live/istiklalcad2hq.stream/Playlist.m3u8")
stream = streams["worst"]

In [ ]:
detection_type = "person"

In [ ]:
if detection_type == "person":
    # initialize the HOG descriptor/person detector
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
    
elif detection_type == "face":
    # Load the cascade for face detector
    cascade = cv2.CascadeClassifier('models/haarcascade_frontalface_default.xml')
    
elif detection_type == "head":
    # Load the cascade for face detector
    cascade = cv2.CascadeClassifier('models/cascadeHead5.xml')

In [ ]:
crop = True

In [ ]:
enum = 0
pnum = 0

In [ ]:
peep = []

In [ ]:
cv2.startWindowThread()

# open webcam video stream
cap = cv2.VideoCapture(stream.url)

# the output will be written to output.avi
out_md = cv2.VideoWriter(
    'output.avi',
    cv2.VideoWriter_fourcc(*'MJPG'),
    15.,
    (640,480))
out_cl = cv2.VideoWriter(
    'clean.avi',
    cv2.VideoWriter_fourcc(*'MJPG'),
    15.,
    (640,480))

to_be_deleted_dc = []

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()


    # resizing for faster detection
    frame = cv2.resize(frame, (640, 480))

    if crop:
        #crop top half
        crop_frame = frame[240:480, 0:640]
        frame = crop_frame

    # make a clean copy of the frame (cv2 images are copied by reference)
    frame_clean = frame.copy()

    if detection_type == "person":

        # using a greyscale picture, also for faster detection
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

        # detect people in the image
        # returns the bounding boxes for the detected objects
        boxes, weights = hog.detectMultiScale(frame, winStride=(8,8) )

    elif detection_type == "face":

        # Convert into grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Detect faces
        boxes = cascade.detectMultiScale(gray, 1.1, 4)

    elif detection_type == "head":

        # Convert into grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Detect faces
        boxes = cascade.detectMultiScale(gray, 1.1, 4)

    boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in boxes])

    for (xA, yA, xB, yB) in boxes:

        # display the detected boxes in the colour picture
        cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 0, 0), 2)

        #get contents of the bounding box
        box = frame_clean[yA:yB, xA:xB]
        
        #how much does this image contain the color of a taxi?
        #this is an attempt to get rid of many yellow taxis that get recognized as people
        taximeter = taxi_score(box, 630)
        if taximeter > 180:
            taxis.append(box)
            continue


        isUnique   = True #until proven not
        isAmbigous = False

        #iterate through the dictionary containing possible duplicates.   
        for key, item in possible_duplicates.items():

            #if duplicate candidate is 2 minutes old, schedule to delete it (can't change dic size during iteration).
            if time.time()-item.birth > 30:
                    to_be_deleted_dc.append(key)
                    
        #iterate through the dictionary containing possible duplicates.   
        for key, item in possible_duplicates.items():

            #if the new box is already in duplicates, don't save it as a unique person.
            similarity = compare(enum, box, item.img)
            if(similarity == "similar"):
                #instead update the duplicate candidate with its newer version.
                possible_duplicates[key] = dupinstance(box)
                isUnique = False
                continue
                
            elif(similarity == "ambigous"):
                isAmbigous = True
                isUnique = False


        # delete old dcs
        for key in to_be_deleted_dc:
            if(key in possible_duplicates.keys()):
                del possible_duplicates[key]

        to_be_deleted_dc.clear()
        

        # if its unique, save it as a unique person.
        if isUnique:
            cv2.imwrite("peep/p{:06d}.jpg".format(pnum), box)
            peep.append(box)
            possible_duplicates[enum] = dupinstance(box)
            enum+=1
            pnum+=1
        
        if isAmbigous:
            #instead update the duplicate candidate with its newer version.
            possible_duplicates[enum] = dupinstance(box)
            enum += 1

    # Write the output video 
    # out_md.write(frame.astype('uint8'))
    out_cl.write(frame_clean.astype('uint8'))
    # Display the resulting frame
    cv2.imshow('mask-detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

            
# When everything done, release the capture
cap.release()
# and release the output
out.release()
# finally, close the window
cv2.destroyAllWindows()
cv2.waitKey(1)